# <center> <font color='#8A0829'> Speech Recognition using Deep Learning</font> </center>

## <font color='#B43104'> Context of the project</font>
Deep learning has revolutionized many fields such as computer vision and speech recognition. In fact, Deep Learning offers more accurate acoustic models and it's all based on neural networks.
In our project we are using Recurrent Neural Networks and Long Short-Term Memory layers. 

<strong>First,</strong> we start with importing different libraries, functions and intializing some variables and parameters.

In [1]:
#  Compatibility imports

from __future__ import absolute_import
from __future__ import division

import time

import tensorflow as tf
import scipy.io.wavfile as wav
import numpy as np
import os
from glob import glob
import librosa

from six.moves import xrange as range

# Importing the mfcc function from python_speech_features
try:
    from python_speech_features import mfcc
except ImportError:
    print("Failed to import python_speech_features.\n Try pip install python_speech_features.")
    raise ImportError


# Constants
SPACE_TOKEN = '<space>'
SPACE_INDEX = 0
FIRST_INDEX = ord('a') - 1  # 0 is reserved to space

# Some configs
num_features = 13
# Accounting the 0th indice +  space + blank label = 28 characters
num_classes = ord('z') - ord('a') + 1 + 1 + 1

# Hyper-parameters
num_epochs = 10
num_hidden = 128  #number of hidden units of the LSTM cell
num_layers = 1    #number of layers in the RNN model
batch_size = 1

initial_learning_rate = 1e-2
#momentum = 0.9

num_examples = 2087 # How much train data we have
num_batches_per_epoch = int(num_examples/batch_size) 

num_test_examples = 487 # How much test data we have
num_test_batches_per_epoch = int(num_test_examples/batch_size)



## <font color='#B43104'> Data Description </font>
The data we are using in our project is the VCTK Corpus. The VCTK Corpus is the Centre for Speech Technology Voice Cloning Toolkit that includes speech data uttered by 109 native speakers of English with various accents. Each speaker reads out about 400 sentences. 
For our project, we used 231 audio files and transcriptions of the words that were spoken in each audio and we took 200 files for the training step and 31 files for the test step.

## <font color='#B43104'> Data pre-processing </font>
We can start preprocessing our data to get the ideal representations that our model needs.
The function below : find_files is used to find the audio and wave files given the data directory.

In [2]:
def find_files(directory, pattern='**/*.wav'):
    """Recursively finds all files matching the pattern."""
    
    return sorted(glob(os.path.join(directory, pattern), recursive=True))

### <center>Pre-processing the audio files</center>
The first step in any automatic speech recognition system is to extract features (identify the components of the audio signal that are good for identifying the linguistic content and discarding all the other stuff which carries information like background noise, emotion etc). 

There are several methods for creating features for machine learning of audio data. The most commonly used is MFCC ( Mel-frequency cepstral coefficients) technique.


In [3]:
# Load input data and extract MFCCs
def load_input_data(directory, pattern):
    
    audio_filename = find_files(directory, pattern='**/*.wav')
    
    inputs = []
    for a in range(len(audio_filename)):
        fs, audio = wav.read(audio_filename[a])
        
       # a = np.lib.pad(audio, (0,(630649 - len(audio))), 'constant', constant_values = (0,0))
        
        MFCC = mfcc(audio, samplerate=fs, nfft=1500)
        
        # Transform MFCC in 3D array
        l = np.asarray(MFCC[np.newaxis,:])
        
        # List of num_examples elements where each element is a 3D array of shape (1, num_frames, num_features=13)
        inputs.append(l)

    return inputs
    

After this transformation the data is stored as a matrix of frequency coefficients over time, and to speed the convergence of the algorithm, the MFCCs are normalized.

In [4]:
# 3D + Normalized MFCCs
def normalized_3D_MFCCs(inputs):
    for i in range(len(inputs)):
        #Normalize
        inputs[i] = (inputs[i] - np.mean(inputs[i]))/np.std(inputs[i])
          #a = np.lib.pad(a, ((0,0),(0,2), (0,0)), 'constant', constant_values = (0, 0))
       # inputs[i] = np.lib.pad(inputs[i],((0,0),(0,(630649 - inputs[i].shape[1])),(0,0)),'constant', constant_values = (0,0))
        
    return inputs

In [5]:
# Sequence_length =  number of frames in each audio file
def seq_length(inputs):
    train_seq_len = []
    for i in range(len(inputs)):
        train_seq_len.append([inputs[i].shape[1]])
        
    return train_seq_len

### <center>Pre-processing the transcriptions</center>
The next step we are doing is to treat our txt files. After reading each txt file, we try to get only the words between [a..z] and replace a space with two spaces so when we split our words based on a space we can still keep the originals spaces in our sentence. In fact, we construct a list of caracters where each caracter is replaced with his code ASCII and the space is replaced with the string 'space'.

In [6]:
#Load target data (txt files)
def load_target_data(directory,pattern):
    
    txt_filename = find_files(directory, pattern='**/*.txt')
    originals = []
    targets = []
    for file in range(len(txt_filename)):
        with open(txt_filename[file], 'r') as f:

            #Only the last line is necessary
            line = f.readlines()[-1]
            
            # Get only the words between [a-z] and replace period for none
            original = ' '.join(line.strip().lower().split(' ')).replace('.', '').replace(':','').replace(',','').replace('?','').replace('-','').replace("'",'').replace('!','').replace(';','').replace('_','')
            originals.append(original)

            target = original.replace(' ', '  ')

            target = target.split(' ')
            
            targets.append(target)
            
    t = []
    for i in range(len(targets)):

        # Adding blank label
        targets[i] = np.hstack([SPACE_TOKEN if x == '' else list(x) for x in targets[i]])

            
        # Transform char into index
        n = np.asarray([SPACE_INDEX if x == SPACE_TOKEN else ord(x) - FIRST_INDEX
                                  for x in targets[i]])
        
        t.append(n)
          
    return originals, t

The ctc_loss function that we are using in our model requires the targets as a sparse representation.

In [7]:
def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape


In [8]:
# Creating sparse representation to feed the placeholder
def sparse_targets(targets):
    train_targets = []
    for i in range(len(targets)):
        train_targets.append(sparse_tuple_from([targets[i]]))
    return train_targets

## <font color='#B43104'>Implementing our speech recognition system </font>

In our implementation, we start by loading our train data and test data using the functions we defined earlier.

In [9]:
#Load and pre-process train data
inputs = load_input_data("C:\\Users\\wafae\\Desktop\\Speech Processing using Deep Learning\\train_data", pattern='*.wav')

train_inputs = normalized_3D_MFCCs(inputs)

train_seq_len = seq_length(train_inputs)

originals, targets = load_target_data("C:\\Users\\wafae\\Desktop\\Speech Processing using Deep Learning\\train_data", pattern='*.txt')

train_targets = sparse_targets(targets)

#Load and pre-process test data
test_inputs = load_input_data("C:\\Users\\wafae\\Desktop\\Speech Processing using Deep Learning\\test_data", pattern='*.wav')

test_inputs_normalized = normalized_3D_MFCCs(test_inputs)

test_originals, test_targets = load_target_data("C:\\Users\\wafae\\Desktop\\Speech Processing using Deep Learning\\test_data", pattern='*.txt')

test_targets = sparse_targets(test_targets)

test_seq_len = seq_length(test_inputs_normalized)

### <center> Defining the variables of our model</center>
For every deep learning model, we should define its variables. 
First, we define inputs, targets and sequence length.

As Deep Learning model we will use LSTM (Long Short-Term Memory) layers, which are a type of recurrent neural network (RNN) architecture useful for modeling data that has long-term sequential dependencies. They are important for time series data because they essentially remember past information at the current time point, which influences their output. This context is useful for speech recognition because of its temporal nature.
<img src="./RNN.png" alt="Image of RNN Architecture" style="height:300px; width:200"/>

So we are defining next, our lstm_cell and our RNN model which implies defining the computation graph : weights, the biases and the activation function and the outputs that our model will get us.
<img src="./LSTM.png" alt="Image of LSTM Architecture" style="height:600px; width:400"/>

In [10]:
# Has size [batch_size, max_stepsize, num_features], but the
# batch_size and max_stepsize can vary along each step
inputs = tf.placeholder(tf.float32, [None, None, num_features])
    
# Here we use sparse_placeholder that will generate a
# SparseTensor required by ctc_loss op.
targets = tf.sparse_placeholder(tf.int32)

# 1d array of size [batch_size]
seq_len = tf.placeholder(tf.int32, [None])

# Defining the cell

lstm_cell = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)

#lstm_cell = tf.contrib.rnn.Dropout(lstm_cell, output_keep=0.9)
    
# Stacking rnn cells
stack = tf.contrib.rnn.MultiRNNCell([lstm_cell] * num_layers,
                                        state_is_tuple=True)


# The second output is the last state and we will no use that
outputs, _ = tf.nn.dynamic_rnn(stack, inputs, seq_len, dtype=tf.float32)
    
shape = tf.shape(inputs)

    
batch_s, max_timesteps = shape[0], shape[1]

# Reshaping to apply the same weights over the timesteps
outputs = tf.reshape(outputs, [-1, num_hidden])

# Truncated normal with mean 0 and stdev=0.1
W = tf.Variable(tf.truncated_normal([num_hidden,
                                         num_classes],
                                        stddev=0.1))


b = tf.Variable(tf.constant(0., shape=[num_classes]))

# Doing the affine projection
logits = tf.matmul(outputs, W) + b

# Reshaping back to the original shape
logits = tf.reshape(logits, [batch_s, -1, num_classes])

# Swap dimensions to Time major for CTC loss
logits = tf.transpose(logits, (1, 0, 2))

### <center> Defining the loss and optimization functions</center>
* To evaluate our model, we will use Connectionist Temporal Classification (CTC) loss function. CTC enables the computation of probabilities of multiple sequences, where the sequences are the set of all possible character-level transcriptions of the speech sample. The network uses the objective function to maximize the probability of the character sequence (i.e., chooses the most likely transcription), and calculates the error for the predicted result compared to the actual transcription to update network weights during training.

* We will use Momentum optimizer which has prooved good performance.



In [11]:
# CTC_LOSS 
loss = tf.nn.ctc_loss(targets, logits, seq_len)

cost = tf.reduce_mean(loss)

#Momentum Optimizer
#optimizer = tf.train.MomentumOptimizer(initial_learning_rate, momentum).minimize(cost)

# Adam Optimizer
optimizer = tf.train.AdamOptimizer(initial_learning_rate).minimize(cost)


decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, seq_len)

# Inaccuracy: label error rate
ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),
                                           targets))

In [12]:
# Training our model using the training data
with tf.Session() as session:
    
    # Initializate the weights and biases
    tf.global_variables_initializer().run()
    # Initialize the model saver
    saver = tf.train.Saver()
    
               
    for curr_epoch in range(num_epochs):
        # Every batch is a wave file
        for batch in range(num_batches_per_epoch):
            train_cost = train_ler = 0
            start = time.time()
            
            
            feed = {inputs: train_inputs[batch],
                            targets: train_targets[batch],
                            seq_len: train_seq_len[batch]}
            
            
            
            batch_cost, _ = session.run([cost, optimizer], feed)
                    
            train_cost += batch_cost*batch_size
                    
            train_ler += session.run(ler, feed_dict=feed)*batch_size
                    

            # Decoding
            d = session.run(decoded[0], feed_dict=feed)

            str_decoded = ''.join([chr(x) for x in np.asarray(d[1]) + FIRST_INDEX])

            # Replacing blank label to none
            str_decoded = str_decoded.replace(chr(ord('z') + 1), '')

            # Replacing space label to space
            str_decoded = str_decoded.replace(chr(ord('a') - 1), ' ')
            print( curr_epoch+1 )
            print('Original:\n%s' % originals[batch])
            print('Decoded:\n%s' % str_decoded)
            

        #train_cost /= num_examples
        #train_ler /= num_examples
        log = "Epoch {}/{}, train_cost = {:.3f}, train_ler = {:.3f}, time = {:.3f}"
        print(log.format(curr_epoch+1, num_epochs, train_cost, train_ler,
                         time.time() - start))
        #Save our model 
        saver.save(session, 'C:\\Users\\wafae\\Desktop\\Speech Processing using Deep Learning\\model\\my-test-model')



1
Original:
please call stella
Decoded:
llclel
1
Original:
ask her to bring these things with her from the store
Decoded:
l
1
Original:
six spoons of fresh snow peas five thick slabs of blue cheese and maybe a snack for her brother bob
Decoded:
l
1
Original:
we also need a small plastic snake and a big toy frog for the kids
Decoded:

1
Original:
she can scoop these things into three red bags and we will go meet her wednesday at the train station
Decoded:

1
Original:
when the sunlight strikes raindrops in the air they act as a prism and form a rainbow
Decoded:

1
Original:
the rainbow is a division of white light into many beautiful colors
Decoded:

1
Original:
these take the shape of a long round arch with its path high above and its two ends apparently beyond the horizon
Decoded:

1
Original:
there is  according to legend a boiling pot of gold at one end
Decoded:

1
Original:
people look but no one ever finds it
Decoded:

1
Original:
when a man looks for something beyond his reach hi

1
Original:
treatment is not an issue with these people
Decoded:

1
Original:
the event was held in the councils headquarters in hamilton
Decoded:

1
Original:
they know no other way
Decoded:

1
Original:
we are just glad to be alive
Decoded:

1
Original:
people will ask about us
Decoded:

1
Original:
however this idea is never developed
Decoded:

1
Original:
its very difficult to find a buyer
Decoded:

1
Original:
it is hard to tell
Decoded:

1
Original:
i think therefore i am 
Decoded:

1
Original:
who was she 
Decoded:

1
Original:
and they were being paid 
Decoded:

1
Original:
it looked like a river
Decoded:

1
Original:
the early goal was a shock to the system for hearts
Decoded:

1
Original:
the occasion was the new labour conference in brighton last year
Decoded:

1
Original:
they have the cash and the fire power
Decoded:

1
Original:
you are a great player
Decoded:

1
Original:
scotland will play its part
Decoded:

1
Original:
he added that he was very sad about o neill
Decode

1
Original:
from that day on we started to look for another full back
Decoded:
t
1
Original:
mr ferguson became a minister after seven years as a journalist
Decoded:
t
1
Original:
it cant just be a cynical marketing exercise
Decoded:
t
1
Original:
it is a matter of the utmost concern to the allies
Decoded:
t
1
Original:
it has failed
Decoded:
t
1
Original:
it was clear
Decoded:
t
1
Original:
it is not for any other purpose
Decoded:
t
1
Original:
of course on tuesday united were beaten despite this
Decoded:
t
1
Original:
is there a new way forward
Decoded:
t
1
Original:
mexico city was a wonderful experience
Decoded:
t
1
Original:
this represents a tough game for us
Decoded:
t
1
Original:
failure to comply with the order is likely to result in prison
Decoded:
t
1
Original:
we need to chat
Decoded:
t
1
Original:
its a blessing
Decoded:
t
1
Original:
but it might have done
Decoded:
t
1
Original:
yesterday he had a chilling warning for the game in scotland
Decoded:
t
1
Original:
it has bee

1
Original:
mr docherty said the whole family was shocked
Decoded:
it
1
Original:
noone has appeared in court in relation to her death
Decoded:
i
1
Original:
i have the first six months of next season to prove myself
Decoded:
i
1
Original:
ive been there
Decoded:
i
1
Original:
i have no sympathy with her at all
Decoded:
i
1
Original:
its a portrait of glasgow not the portrait
Decoded:
i
1
Original:
it is not fair
Decoded:
i
1
Original:
it just came into my head that day
Decoded:
i
1
Original:
i had a train to catch
Decoded:
i
1
Original:
any change would be subject to the scottish parliaments approval
Decoded:
i
1
Original:
the whole process is a vicious circle at the moment
Decoded:
i
1
Original:
suddenly the rugby world had changed
Decoded:
i
1
Original:
treasury spokesman malcolm bruce is also considered a potential contender
Decoded:
i
1
Original:
people will ask about us
Decoded:
i
1
Original:
the appeal tribunal will give its decision at a later date
Decoded:
i
1
Original:
it is 

1
Original:
craig is a major concern for us
Decoded:
t
1
Original:
the home team had an amazing start
Decoded:
i
1
Original:
my body is clearly capable
Decoded:
i
1
Original:
referees are easy targets
Decoded:
i
1
Original:
he said she was mad
Decoded:
i
1
Original:
we are in the end game
Decoded:
i
1
Original:
i am very happy to be here
Decoded:
i
1
Original:
the airport was even different
Decoded:
i
1
Original:
we do the same
Decoded:
i
1
Original:
thats the law of the land
Decoded:
i
1
Original:
but no such amendment has been brought before the scottish parliament
Decoded:
i
1
Original:
the weekly average was three hours
Decoded:
i
1
Original:
donald was his friend
Decoded:
t
1
Original:
i am so upset
Decoded:
i
1
Original:
it was no secret that i wanted to sign him
Decoded:
i
1
Original:
on thursday mr blair will address the institute of public policy research
Decoded:
t
1
Original:
the boat was fantastic
Decoded:
i
1
Original:
it is a time bomb
Decoded:
i
1
Original:
id like to ta

...

10
Original:
its about my time as a guitarist
Decoded:
ia bomatamsi gotasi
10
Original:
there is no other solution to congestion in edinburgh
Decoded:
thes saniconto can gatanmanvhadombraa
10
Original:
its not even finished
Decoded:
its etenfenah
10
Original:
it should be equal
Decoded:
io beeo
10
Original:
cruelty is an accepted feature of conflict in the balkans
Decoded:
oltes vhe atjie taono tonhe bo clons
10
Original:
we are very strongly behind the bid
Decoded:
we vohroainbenthe bet
10
Original:
in addition certain areas are closed at certain times
Decoded:
i n deioncsan atato asamtaans
10
Original:
i am very happy to be here
Decoded:
ien go a bhetobe 
10
Original:
it was no secret that i wanted to sign him
Decoded:
is esogo theawottin in
10
Original:
on thursday mr blair will address the institute of public policy research
Decoded:
inthesap wesa beweas the ans oho def pibet boesosapt
10
Original:
it could have gone either way
Decoded:
i a gonethewa
10
Original:
he was then replac

10
Original:
if the red of the second bow falls upon the green of the first the result is to give a bow with an abnormally wide yellow band since red and green light when mixed form yellow
Decoded:
i therat a asainbefos a pon thegrorin o ohe fe theos aissigee bo was the ane n oia wattenobantincs fo an rotoonifom ia
10
Original:
this is a very common type of bow one showing mainly red and yellow with little or no green or blue
Decoded:
thes is veconena bo wonioon weiesine o  whahotonogreno tlei
10
Original:
it is on the internet
Decoded:
i thes on tinoo
10
Original:
smoking is also very important
Decoded:
snecing isosafenpodtin
10
Original:
he has the courage of the unaware
Decoded:
he hesocoobo the we noo
10
Original:
life was like a timetable
Decoded:
thethisak giomi wa
10
Original:
it shouldnt come as a surprise but it does
Decoded:
iid encom wot aso osepros ble ides
10
Original:
nonetheless the overall picture is healthy
Decoded:
nowetetheiothorepetoei
10
Original:
im so happy that 

10
Original:
this is a successful outcome
Decoded:
thesis asiaso ao
10
Original:
suddenly the fir park side were in disarray
Decoded:
sili thefepyta wan disare
10
Original:
she died at her home
Decoded:
hegidaa ha
10
Original:
alice the girl was said by doctors to be in good condition
Decoded:
as the ga wasapyda estebencradcondiion
10
Original:
five minutes later the home team were back in front
Decoded:
iv  esethe the hame w byifront
10
Original:
it was a piece of art
Decoded:
i wasapesav vat
10
Original:
i would relish the prospect
Decoded:
i waroa pepro pet
10
Original:
these men are a part of india
Decoded:
thesmana ad padaen be
10
Original:
i was caught out a little bit
Decoded:
i wascotatadita bet
10
Original:
the petition was formally presented at downing street yesterday
Decoded:
thepe dihonmmwasfoopresantet ad aingshreis te i
10
Original:
i have wanted to buy one for the national gallery
Decoded:
i vwamie de bamfathena nea
10
Original:
it has to be an event
Decoded:
ipastibena

10
Original:
it had so much to do with output
Decoded:
idthasa wocode wa vopot
10
Original:
must have been at the bar
Decoded:
ma hav bemi a tepbo
10
Original:
she died in middle age
Decoded:
o dadi in a vo
10
Original:
no evidence that it was osama bin laden
Decoded:
oo din thot itastasopi in
10
Original:
they need to win a trophy
Decoded:
thedetoanatroe
10
Original:
half the young people had had contact with the police
Decoded:
a theae bohahatconoi a lo
10
Original:
it is possible to be too good
Decoded:
hi tisplesa bebeteaid
10
Original:
in many ways that is as important
Decoded:
imani s tha  is as amton
10
Original:
here is a question for you
Decoded:
hia ris icoonfi
10
Original:
i was just a kid
Decoded:
i wesstecet
10
Original:
sadly it sounds as if it is
Decoded:
sia ions oas oifi idos
10
Original:
hospitals are for the ill
Decoded:
aspetos afo the
10
Original:
however he is a coach and he remains a coach at heart
Decoded:
hao heis i co ante renmensi coio ha t
10
Original:
emplo

10
Original:
we think all other measures are not exhausted
Decoded:
we then gco oeonotes  anotasosto d
10
Original:
there was no time scale
Decoded:
the was no tansi
10
Original:
last night he pledged to clear his name
Decoded:
wes not he bidyles the
10
Original:
on thursday mr blair will address the institute of public policy research
Decoded:
hothestammastobe wao drosinoidopobleposyyseh
10
Original:
he is not very big either
Decoded:
isnofobe cathe
10
Original:
i have a moral dilemma
Decoded:
i ha amo thaae
10
Original:
even the one she loved
Decoded:
ifon the wonho eaf d
10
Original:
you have no chance of planning permission
Decoded:
ye haenasaploanemion
10
Original:
we are encouraged by the news
Decoded:
we an copahenes
10
Original:
in addition certain areas closed at certain times
Decoded:
in a deionsanascostesanas
10
Original:
the professor said the results were positive
Decoded:
the befasasadherasawa posati
10
Original:
in fact it is deeply unpleasant
Decoded:
infit i tosde benp

### <center> Testing our model</center>
To test our model, we took 31 test files. This time we will only provide our model with the same batch size as we did in the training meaning a wave file at a time. And when we run our model we will not provide him the optimizer function.

In [13]:
# Test our model             
def test(session, model_path):
    
    # Restor our model
    saver.restore(session, model_path)
    print("Model restored from file: %s" % model_path)
    
    for batch in range(num_test_batches_per_epoch):
        
        test_feed = {inputs: test_inputs_normalized[batch],
                            targets: test_targets[batch],
                            seq_len: test_seq_len[batch]}

        test_cost, test_ler = session.run([cost, ler], feed_dict=test_feed)

        # Decoding
        d = session.run(decoded[0], feed_dict=test_feed)

        str_decoded = ''.join([chr(x) for x in np.asarray(d[1]) + FIRST_INDEX])

        # Replacing blank label to none
        str_decoded = str_decoded.replace(chr(ord('z') + 1), '')

        # Replacing space label to space
        str_decoded = str_decoded.replace(chr(ord('a') - 1), ' ')

        print('Test_Original:\n%s' % test_originals[batch])
        print('Test_Decoded:\n%s' % str_decoded)
        log = "test_cost = {:.3f}, test_ler = {:.3f}"
        print(log.format( test_cost, test_ler))
        
    return str_decoded


with tf.Session() as session:
    a = test(session, 'C:\\Users\\wafae\\Desktop\\Speech Processing using Deep Learning\\model\\my-test-model')
    print(a)
    

INFO:tensorflow:Restoring parameters from C:\Users\wafae\Desktop\Speech Processing using Deep Learning\model\my-test-model
Model restored from file: C:\Users\wafae\Desktop\Speech Processing using Deep Learning\model\my-test-model
Test_Original:
funding for the science centre has come from a number of sources
Test_Decoded:
fanethethesasamsaehiscomomeneosas
test_cost = 156.117, test_ler = 0.703
Test_Original:
it is a court case
Test_Decoded:
i is cosos
test_cost = 27.584, test_ler = 0.556
Test_Original:
there is no plan b
Test_Decoded:
the es npam e
test_cost = 30.367, test_ler = 0.444
Test_Original:
the court was shown a video of the house on that day
Test_Decoded:
the co ishomi viu wo theasolade
test_cost = 88.547, test_ler = 0.577
Test_Original:
he will go a long way
Test_Decoded:
o  galono
test_cost = 48.078, test_ler = 0.667
Test_Original:
it is just a tax on employment
Test_Decoded:
be  osasemonemt
test_cost = 70.609, test_ler = 0.700
Test_Original:
there is no other solution to co

Test_Original:
id just like to play
Test_Decoded:
iashatlo
test_cost = 35.991, test_ler = 0.800
Test_Original:
because we do not need it
Test_Decoded:
pe cus e deno ndat
test_cost = 37.022, test_ler = 0.480
Test_Original:
there was a rush of water
Test_Decoded:
be es amahre vsa
test_cost = 45.841, test_ler = 0.720
Test_Original:
last night mr robertson said he was delighted
Test_Decoded:
plet o nsho les ansaees dled
test_cost = 105.378, test_ler = 0.689
Test_Original:
that deal is a joke
Test_Decoded:
i dells ja 
test_cost = 38.770, test_ler = 0.632
Test_Original:
the premier league has been a huge success
Test_Decoded:
the prenle es benueeca
test_cost = 91.291, test_ler = 0.571
Test_Original:
if we cannot succeed in this endeavour then so be it
Test_Decoded:
e e ellsitse n eson da ve pense bet
test_cost = 115.824, test_ler = 0.615
Test_Original:
it carried a double message
Test_Decoded:
iodobblemaied
test_cost = 63.421, test_ler = 0.704
Test_Original:
that means an extra two years
Tes

Test_Original:
what kind of man does that mr dick 
Test_Decoded:
we omenand es ta  iste di 
test_cost = 93.958, test_ler = 0.611
Test_Original:
so did she meet him 
Test_Decoded:
a dehonetim
test_cost = 39.101, test_ler = 0.667
Test_Original:
so what is the campaign about 
Test_Decoded:
e tes theaman a et
test_cost = 59.981, test_ler = 0.548
Test_Original:
and does it 
Test_Decoded:
en tos at
test_cost = 29.877, test_ler = 0.538
Test_Original:
before the game we went for a pint
Test_Decoded:
eco the genen repit t
test_cost = 75.494, test_ler = 0.559
Test_Original:
we needed financial backing to get it off the ground
Test_Decoded:
we feada baun ga ef rod
test_cost = 103.354, test_ler = 0.654
Test_Original:
youd better believe it
Test_Decoded:
 ad a beli
test_cost = 52.247, test_ler = 0.682
Test_Original:
new year will be very busy
Test_Decoded:
nbe ve besi
test_cost = 62.347, test_ler = 0.654
Test_Original:
theres still plenty time left
Test_Decoded:
tes eplen eton
test_cost = 61.810, t

Test_Original:
but one was not enough
Test_Decoded:
be wamas teno
test_cost = 49.597, test_ler = 0.591
Test_Original:
i have spoken to head office
Test_Decoded:
os becind i he oes
test_cost = 66.842, test_ler = 0.750
Test_Original:
i am not ready to walk away
Test_Decoded:
inaretaqo o
test_cost = 70.958, test_ler = 0.741
Test_Original:
the shooting created panic in the street
Test_Decoded:
theheonre o paliinshret
test_cost = 78.729, test_ler = 0.550
Test_Original:
so many people will be affected by this
Test_Decoded:
se ie wu beetebo as
test_cost = 102.373, test_ler = 0.641
Test_Original:
as many as the market will support
Test_Decoded:
ismnans thea ilspo et
test_cost = 69.774, test_ler = 0.559
Test_Original:
they need to look after themselves
Test_Decoded:
thenthe goiea
test_cost = 88.055, test_ler = 0.765
Test_Original:
this is the time to use it
Test_Decoded:
tis thesonosi
test_cost = 49.887, test_ler = 0.615
Test_Original:
you know the sort of thing
Test_Decoded:
yea salsen
test_co

Test_Original:
he admitted that he knew nothing about it
Test_Decoded:
emeo tenafena a e
test_cost = 101.480, test_ler = 0.732
Test_Original:
it seems a shame to leave it
Test_Decoded:
esentojon d ile ve
test_cost = 61.454, test_ler = 0.750
Test_Original:
it was the climax of the thing
Test_Decoded:
i esacounaef thefe
test_cost = 59.321, test_ler = 0.667
Test_Original:
the concerns are the same
Test_Decoded:
the cemsens athsan
test_cost = 39.955, test_ler = 0.440
Test_Original:
i cant get a job
Test_Decoded:
i lan gas o job p
test_cost = 29.995, test_ler = 0.438
Test_Original:
and they were being paid 
Test_Decoded:
em e benlad
test_cost = 67.590, test_ler = 0.654
Test_Original:
was everything done to save people 
Test_Decoded:
ws ave en etsepepo
test_cost = 72.479, test_ler = 0.611
Test_Original:
what was the matter for concern 
Test_Decoded:
we lis theasafegense
test_cost = 67.104, test_ler = 0.667
Test_Original:
do i think about them often 
Test_Decoded:
dev enu byllo e
test_cost = 

Test_Original:
the problems began a long time ago
Test_Decoded:
the prob ls be alntlami ga
test_cost = 63.176, test_ler = 0.441
Test_Original:
i am not sure about the rate of progress
Test_Decoded:
i ohre ma e a preugre
test_cost = 85.965, test_ler = 0.625
Test_Original:
to beat the european champion is quite nice
Test_Decoded:
be eipenhomeunsa
test_cost = 104.374, test_ler = 0.744
Test_Original:
in the old days it was easy
Test_Decoded:
dineldaseeseo
test_cost = 64.793, test_ler = 0.704
Test_Original:
however the groups denied the claims
Test_Decoded:
a lethe gateapedla
test_cost = 86.413, test_ler = 0.694
Test_Original:
she is now on the back benches
Test_Decoded:
hes namleba beniels
test_cost = 59.946, test_ler = 0.567
Test_Original:
i think its about time
Test_Decoded:
hi ende baa
test_cost = 54.645, test_ler = 0.818
Test_Original:
the airport was even different
Test_Decoded:
te e plet wes evendaram
test_cost = 59.147, test_ler = 0.500
Test_Original:
there was no time scale
Test_De

Test_Original:
it was a bank holiday after all
Test_Decoded:
iis ben loll trewa
test_cost = 66.756, test_ler = 0.645
Test_Original:
ms mcneill was killed on impact
Test_Decoded:
wes no es olldlmanplat d
test_cost = 69.444, test_ler = 0.742
Test_Original:
there was an element of frustration
Test_Decoded:
thels nan fehrohie
test_cost = 67.933, test_ler = 0.686
Test_Original:
i should think so too
Test_Decoded:
iie tia e
test_cost = 39.211, test_ler = 0.762
Test_Original:
how do you take them away 
Test_Decoded:
aea tlea
test_cost = 65.371, test_ler = 0.815
Test_Original:
after the match do you 
Test_Decoded:
aethead je
test_cost = 52.893, test_ler = 0.667
Test_Original:
i will always remember the horrible images
Test_Decoded:
ieionamle vehob bellanejeso
test_cost = 100.404, test_ler = 0.738
Test_Original:
they are looking for profit
Test_Decoded:
telauwe pofet
test_cost = 69.879, test_ler = 0.704
Test_Original:
i remain a buyer of the shares
Test_Decoded:
aanebofehes
test_cost = 66.297, 